In [5]:
class Power():
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent

class Number():
    def __init__(self,number):
        self.number = number

class Variable():
    def __init__(self,symbol):
        self.symbol = symbol

Power(Variable("x"),Number(2))
##model of the expression x^2

class Product():
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

#3x^2:
Product(Number(3),Power(Variable("x"),Number(2)))

class Sum():
    def __init__(self, *exps):
        self.exps = exps

class Function():
    def __init__(self,name):
        self.name = name

class Apply():
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument

f_expression = Product(
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                        Variable("x")),
                    Apply(
                        Function("sin"),
                        Variable("x")))

In [8]:
#ex10.2
import math
def f(y,z):
    return math.log(y**z)

Apply(Function("ln"), Power(Variable("x"), Variable("z")))

In [12]:
class Quotient():
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator

Quotient(Sum(Variable("a"),Variable("b")),Number(2))

In [13]:
class Difference():
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2

Difference(
    Power(Variable("b"),Number(2)),
    Product(Number(4),Product(Variable("a"),Variable("c"))))

In [14]:
class Negative():
    def __init__(self,exp):
        self.exp = exp

Negative(Sum(Power(Variable("x"),Number(2)),Variable("y")))

In [16]:
from abc import ABC, abstractmethod

class Expression(ABC):
    @abstractmethod
    def latex(self):
        pass
    def _repr_latex_(self):
        return "$$" + self.latex() + "$$"
    @abstractmethod
    def evaluate(self, **bindings):
        pass
    @abstractmethod
    def substitute(self, var, expression):
        pass
    @abstractmethod
    def expand(self):
        pass
    @abstractmethod
    def display(self):
        pass
    def __repr__(self):
        return self.display()
    @abstractmethod
    def derivative(self,var):
        pass
    
    def __call__(self, *inputs):
        var_list = sorted(distinct_variables(self))
        return self.evaluate(**dict(zip(var_list, inputs)))
    
    def __add__(self, other):
        return Sum(self,package(other))
    
    def __sub__(self,other):
        return Difference(self,package(other))
    
    def __mul__(self,other):
        return Product(self,package(other))
    
    def __rmul__(self,other):
        return Product(package(other),self)
    
    def __truediv__(self,other):
        return Quotient(self,package(other))
    
    def __pow__(self,other):
        return Power(self,package(other))
    
    @abstractmethod
    def _python_expr(self):
        pass
    
    def python_function(self,**bindings):
#         code = "lambda {}:{}".format(
#             ", ".join(sorted(distinct_variables(self))),
#             self._python_expr())
#         print(code)
        global_vars = {"math":math}
        return eval(self._python_expr(),global_vars,bindings)